In [30]:
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import random
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import datetime
import sys
import json
import jsonlines
import pprint
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager


In [2]:
# 새로 업로드한 HTML 파일을 읽기
with open('Factiva.com.html', 'r', encoding='utf-8') as file:
    new_html_content = file.read()

# HTML 내용을 파싱
soup_new = BeautifulSoup(new_html_content, 'html.parser')

In [12]:
# # <p> 태그를 사용해 기사 내용을 찾아보기
# paragraphs = soup_new.find_all('p')

# # 기사 내용이 포함될 것으로 예상되는 긴 <p> 태그들을 찾기
# long_paragraphs = [p for p in paragraphs if len(p.get_text(strip=True)) > 50]

# # 확인을 위해 첫 번째로 발견된 긴 <p> 태그의 내용 출력
# long_paragraphs[0].get_text(strip=True) if long_paragraphs else "No long paragraphs found"


'Reuters News,   Wednesday,  06  September  2023,  01:19 GMT,  129 Words,  (English), Copyright 2023 Thomson Reuters. All Rights Reserved.(Document LBA0000020230905ej9503js1)'

In [3]:
paragraphs

NameError: name 'paragraphs' is not defined

In [4]:
# 각 <p> 태그의 내용을 분석하여 기사의 제목과 내용 추출
titles_new = []
contents_new = []

for p in long_paragraphs:
    text = p.get_text(strip=True)
    
    # 기사의 제목과 내용을 분리하기 위한 간단한 방법: 
    # 첫 번째 문장을 제목으로 간주하고 나머지를 본문으로 간주
    sentences = text.split('.')
    titles_new.append(sentences[0])
    contents_new.append(".".join(sentences[1:]))

# 결과를 DataFrame 형식으로 변환
df_new = pd.DataFrame({
    'Title': titles_new,
    'Content': contents_new
})

# DataFrame의 첫 5개 행을 출력하여 확인
df_new.head()


NameError: name 'long_paragraphs' is not defined

In [5]:
# Read the new text file content
with open("Factiva-07September20231123.txt", "r", encoding="ISO-8859-1") as file:
    new_txt_content = file.read()

In [6]:
# Final code to split the articles based on the "Document XXXX..." pattern, keeping the delimiter at the end,
# and then remove the "Document XXXX..." pattern from the start of each article

# Split the content based on the "Document XXXX..." pattern, keeping the delimiter at the end
articles_with_document_split = re.split(r'(Document \w{8}\d{8}\w{7})', new_txt_content)

# Adjust the split to form articles with the "Document XXXX..." pattern only at the end
adjusted_articles_final = [articles_with_document_split[i+1] + articles_with_document_split[i] 
                          for i in range(0, len(articles_with_document_split)-1, 2)]

# Convert the adjusted articles into a DataFrame
df_adjusted_articles_final = pd.DataFrame(adjusted_articles_final, columns=["Article_Content"])

# Remove the "Document XXXX..." pattern from the start of each article
df_adjusted_articles_final["Article_Content"] = df_adjusted_articles_final["Article_Content"].str.replace(r'^Document \w{8}\d{8}\w{7}', '', regex=True)

df_adjusted_articles_final["Article_Content"] = df_adjusted_articles_final["Article_Content"].str.replace(r'^.*?\n+', '', regex=True)


In [12]:
df_adjusted_articles_final['Article_Content'][0]

'Business\nDigital currencies: CBA to limit fund transfers in crypto scam crackdown\n\nDominic Powell \n485 words\n9 June 2023\nThe Sydney Morning Herald\nSMHH\nFirst\n26\nEnglish\n? 2023 Copyright John Fairfax Holdings Limited. www.smh.com.au\n\nCommonwealth Bank will limit customers from sending money to certain cryptocurrency exchanges and implement monthly limits on crypto purchases in an effort to crack down on scams and fraud.\n\nYesterday, the big four bank introduced a raft of new scam prevention methods, including bans or 24-hour holds on payments to certain crypto exchanges, becoming the second major bank to do so after Westpac took similar action last month.\n\nCBA would not identify the exchanges that would be limited under the new system. However, it\'s likely Binance - one of the world\'s largest and most popular cryptocurrency exchanges - will be included.\n\nThis week, US regulators took action against Binance and Coinbase, another major exchange, accusing them of viola

In [12]:
# Extract title, content, and publication date from each article

# Extract title (assuming title is the first line of the article)
df_adjusted_articles_final['Title'] = df_adjusted_articles_final['Article_Content'].str.split('\n').str[0]

# Extract content (assuming content is everything after the title and before the last line which contains the Document pattern)
df_adjusted_articles_final['Content'] = df_adjusted_articles_final['Article_Content'].apply(lambda x: '\n'.join(x.split('\n')[1:-1]))

# Extract publication date from the "Document XXXX..." pattern
df_adjusted_articles_final['Publication_Date'] = df_adjusted_articles_final['Article_Content'].str.extract(r'Document \w{4}(\d{8})\w{7}')

# Drop the original combined column
df_extracted_data = df_adjusted_articles_final.drop(columns=["Article_Content"])

# Display the first few rows of the extracted DataFrame
df_extracted_data.head()


,Title,Content,Publication_Date
0,Business,Digital currencies: CBA to limit fund transfer...,NaN
1,General News,Winter is here for exchanges\n\nLex comment \n...,NaN
2,Business,CBA s crypto block to hit scammers\n\n144 wor...,NaN
3,Business,"Greatest innovation ever: AI to slash prices, ...",NaN
4,News,Banks put the brake on crypto\n\nBen Butler \n...,NaN


In [38]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Send POST request using Python's requests library
response = requests.post(NEWS_LIST_URL, headers=NEWS_LIST_HEADERS, json=NEWS_LIST_PAYLOAD)

# If you need to interact with the resulting page using Selenium:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Save the response content to a temporary file
with open("temp_response.html", "w", encoding="utf-8") as f:
    f.write(response.text)

# Use Selenium to navigate to the saved file
driver.get("file:temp_response.html")


# Interact with the page using Selenium if needed...

# If you just need to parse the content:
soup = BeautifulSoup(response.text, 'html.parser')
# Extract necessary data from soup...

# Close the WebDriver session
driver.quit()


In [39]:
soup

<body>
<form action="https://global.factiva.com/factivalogin/login.asp?p=snp&amp;productname=cyclone&amp;landingpage=https%3a%2f%2fglobal.factiva.com%2fredir%2fdefault.aspx&amp;HTTP_REFERER=global.factiva.com&amp;returnedfromloginserver=success" id="login" method="post" name="login">
</form></body>

<script type="text/javascript">document.forms['login'].submit()</script>


In [17]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

service = Service('C:/Users/user/chromedriver/chromedriver.exe')
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# 대상 웹사이트로 이동
driver.get("https://snapshot.factiva.com/Search/SSResults")

# 30번 반복
for i in range(30):
    # .article-view-trigger 클래스를 가진 a태그를 찾음
    articles = driver.find_elements(By.CSS_SELECTOR, ".article-view-trigger")
    
    # 첫 번째 항목 클릭
    articles[0].click()
    
    # 페이지 로딩을 기다림 (필요에 따라 시간 조정 가능)
    time.sleep(5)
    
    # 뒤로 가기
    driver.back()
    
    # 페이지 로딩을 기다림 (필요에 따라 시간 조정 가능)
    time.sleep(5)

# 작업 완료 후 웹 드라이버 종료
driver.quit()

IndexError: list index out of range

In [21]:
NEWS_LIST_URL = "https://snapshot.factiva.com/Search/SSResults"

NEWS_LIST_PAYLOAD = {
    '_XFORMSTATE': '',
    '_XFORMSESSSTATE': 'TXswOnsyOnswOjAsMjoiTlAiLDM6W10sNDoiZW4iLDU6MCw5OiJHTCJ9LFU6Ii9QYWdlcy9JbmRleCJ9fREAADwAAAA=',
    'xsrftoken': 'FGHcwqemEekdIMQEAiafLn_Memk9_fdcOx2HHkkF51WrL40ywDDBrvfInJxCqXYxQ70RAIHi-MxYWC5m_Fsi4lHfQyI1',
    'ins': '1',
    'napc': 'NS',
    'searchModel': '{"pid":"Np_V1_685648_685648_020102","pname":"South Korea","ptype":"User","searchFilter":{"ft":"cryptocurrency","hch":"All","dt":"Last3Months","ism":false,"frtr":0,"sfc":{"code":"","desc":"cryptocurrency"},"sl":{"slid":"All","slds":"false","sljson":"[]","sln":"All Sources"}},"ReturnUrl":""}'
}


NEWS_LIST_HEADERS = {
'Accept':
'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
'Accept-Encoding':
'gzip, deflate, br',
'Accept-Language':
'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
'Cache-Control':
'max-age=0',
'Content-Length':
'751',
'Content-Type':
'application/x-www-form-urlencoded',
'Cookie':
'persistent=lang=en; maintenance=mode=1; GPLogin=FP%5FA=9SUN009900&FP%5FN=16&FP%5FU=sungeu; s_ecid=MCMID%7C53844123662998131672474797158983060138; djm="id=55858978-18d5-4a75-a781-b9c1dfee296b"; utag_main=v_id:018ac0d33f490019216047626e370506f002a06700bd0$_sn:1$_se:4$_ss:0$_st:1695454017529$ses_id:1695452184394%3Bexp-session$_pn:2%3Bexp-session$_prevpage:DJCSS_CustomerResource_FAQ_Search%20Results%3Bexp-1695455817530$vapi_domain:factiva.com; GSLogin=GL%5FI=eyJhbGciOiJSUzI1NiIsImtpZCI6IjJEN0IwQTFERkJCNzlDRDFBQjM4NzNCMTcyODMyRjkxMENEQkRBREIiLCJ0eXAiOiJKV1QifQ%2EeyJwaWIiOnsiYWNjb3VudF9pZCI6IjlTVU4wMDk5MDAiLCJhbGxvd19hdXRvX2xvZ2luIjoidHJ1ZSIsImFwYyI6IjkiLCJjdGMiOiJEIiwibWFzdGVyX2FwcF9pZCI6InpnUWtOaVI5REJxYW1aQnFJaTZEME5hSXZ6TEtraDNxIiwic2Vzc2lvbl9pZCI6IjI3MjA0WHhYX01ZNERBT0pRR0pSR0VNQlFHUTNUSVpERk1FNERLTUpXR0ZRVEdZM0RHQlJHR01URE1FNEEiLCJlbWFpbCI6ImRvbm90LmVkaXRAY28uY29tIn0sImlzcyI6Imh0dHBzOi8vYWNjb3VudHMuZG93am9uZXMuY29tL29hdXRoMi92MiIsImVtYWlsIjoiZG9ub3QuZWRpdEBjby5jb20iLCJzdWIiOiJhdXRoMHxEM1daWUo3UzVLQTJGQ1NYUE9HQlZIWFBQWSIsImF1ZCI6InpnUWtOaVI5REJxYW1aQnFJaTZEME5hSXZ6TEtraDNxIiwiYXpwIjoiemdRa05pUjlEQnFhbVpCcUlpNkQwTmFJdnpMS2toM3EiLCJpYXQiOjE2OTU3MTU3NjksImV4cCI6MTY5NTk3NDk2OX0%2EWPCtIcXZX%2DIk47EewN%2DI8IB%2D9zKkPXlJFxN7rhefMvJeCN12IsOl1yENZUYmb5PWcRKxMQHpzQQGMhcKYkmAJtpKxAQuG1sU7qODryMfQICEmZkf88eBzNUUecdeoqCtUgOJyny%5FaLPw73Xu41t%2D6oTDSayCMvx4xbY0xfosjrWxgYK503UYoSU68c2813jCA6c0b0O5Hr7seOPAmUyGCgkKeBHqEPjErKlBiirx5vchkAQBgvgTlQV7UZATzT5LDIJyiNn5t5Hfch14VWFNDy%2DaEA0p85JrjMyAeoX9f%2Don65NFI7VJi%2Daw%5FVziaZyEfyfiIFktO%2DkS6bzj%5FfFn1A&GL%5FA=9SUN009900&FP%5FA=9SUN009900&GL%5FU=sungeu&GL%5FN=16&GL%5FEU=D3WZYJ7S5KA2FCSXPOGBVHXPPY&FP%5FEU=D3WZYJ7S5KA2FCSXPOGBVHXPPY&FP%5FN=16&FP%5FU=sungeu&GL%5FS=27204XxX%5FMY4DAOJQGJRGEMBQGQ3TIZDFME4DKMJWGFQTGY3DGBRGGMTDME4A&FP%5FS=27204XxX%5FMY4DAOJQGJRGEMBQGQ3TIZDFME4DKMJWGFQTGY3DGBRGGMTDME4A; LPLogin=PDRREFS=GL; GSHL=RS=11; DJISML=; xsrftoken=k-pIWV2S-jTifIjiqQ4QDhVrc7zutbObAeT3Yt2TMfumhSpNGWa01lTmJpsz5l3RsYod51HGf2wodB2LsUoDA768e6I1; AMCVS_CB68E4BA55144CAA0A4C98A5%40AdobeOrg=1; AMCV_CB68E4BA55144CAA0A4C98A5%40AdobeOrg=1585540135%7CMCIDTS%7C19627%7CMCMID%7C53844123662998131672474797158983060138%7CMCAAMLH-1696320576%7C11%7CMCAAMB-1696320576%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1695722976s%7CNONE%7CMCAID%7CNONE%7CvVersion%7C4.4.0; s_cc=true; RT="z=1&dm=factiva.com&si=ks4wc01qzkk&ss=ln01e0dk&sl=2&tt=bn8&bcn=https%3A%2F%2Fapi.dowjones.com%2Frealusermetrics%2Fmetrics%2Fuser&ld=kog&ul=xll"',
'Origin':
'https://snapshot.factiva.com',
'Referer':
'https://snapshot.factiva.com/Pages/Index',
'Sec-Ch-Ua':
'"Google Chrome";v="117", "Not;A=Brand";v="8", "Chromium";v="117"',
'Sec-Ch-Ua-Mobile':
'?0',
'Sec-Ch-Ua-Platform':
"Windows",
'Sec-Fetch-Dest':
'document',
'Sec-Fetch-Mode':
'navigate',
'Sec-Fetch-Site':
'same-origin',
'Sec-Fetch-User':
'?1',
'Upgrade-Insecure-Requests':
'1',
'User-Agent':
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
}

response = requests.post(NEWS_LIST_URL, headers=NEWS_LIST_HEADERS, data=NEWS_LIST_PAYLOAD)

In [26]:
response = requests.post(NEWS_LIST_URL, headers=NEWS_LIST_HEADERS, data=NEWS_LIST_PAYLOAD)

In [22]:
response

<Response [200]>

In [27]:
response.text

'<!doctype html>\r\n<!--[if lt IE 7 ]> <html lang="en" class="no-js ie ie6"> <![endif]-->\r\n<!--[if IE 7 ]>    <html lang="en" class="no-js ie ie7"> <![endif]-->\r\n<!--[if IE 8 ]>    <html lang="en" class="no-js ie ie8"> <![endif]-->\r\n<!--[if IE 9 ]>    <html lang="en" class="no-js ie ie9"> <![endif]-->\r\n<!--[if IE 10 ]>    <html lang="en" class="no-js ie ie10"> <![endif]-->\r\n<!--[if (gt IE 10)|!(IE)]><!-->\r\n<html lang="en" class="no-js">\r\n<!--<![endif]-->\r\n<head>\r\n    <title>Factiva</title>\r\n    <meta charset="utf-8" />\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />\r\n    <link rel="icon" type="image/png" href="/Content/images/favicon.png?v=60.31.0" />\r\n    <link rel="apple-touch-icon" href="/Content/images/favicon.png?v=60.31.0" />\r\n    <meta name="viewport" content="width=1016" />\r\n    <script type="text/javascript" src="/Content/js/modernizr.js?v=60.31.0"></script>\r\n\r\n    <script type="text/javascript">\r\n//<![CDATA[\r\nvar req

In [24]:
print("Response Content:", response.text[10000:10500])  # 응답 내용의 첫 500자 출력                             

Response Content: 
